In [ ]:
import moth
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import scipy
import scipy.signal
import h5py

PATH = '/storage/home/adz6/group/project'
DIGDATAPATH = os.path.join(PATH, 'datasets', 'data', 'lab', 'dig')
VNACORRECTION = os.path.join(PATH, 'results', 'moth', 'vna_correction_files')

#DAQDATAPATH = os.path.join(PATH, 'daq_data')
#MOTHPATH = os.path.join(PATH, 'moth')
#MOTHDATA = os.path.join(MOTHPATH, 'data/datasets')
#PLOTPATH = os.path.join(PATH, 'plots/moth')
#MOTHRESULTS = os.path.join(PATH, 'results/moth')


def SumData(array_data, xpos, ypos, nch, r=0.1, f=25.8e9, antispiral=False):
    
    xx, yy = np.meshgrid(xpos, ypos)
    array_antenna_theta = np.arange(0, 2 * np.pi, 2 * np.pi / array_data.shape[0])

    #xx, yy = np.expand_dims(xx, axis=0).repeat(array_data['angle'].size, axis=0), np.expand_dims(yy, axis=0).repeat(array_data['angle'].size, axis=0)

    xantenna = (r * np.cos(-1 * array_antenna_theta)).reshape((array_antenna_theta.size, 1, 1))
    yantenna = (r * np.sin(-1 * array_antenna_theta)).reshape((array_antenna_theta.size, 1, 1))
    
    xdist = xantenna - xx.reshape((1, *xx.shape))
    ydist = yantenna - yy.reshape((1, *yy.shape))
    
    dist_grid = np.sqrt(xdist ** 2 + ydist ** 2)
    
    amplitude_weight = 1 / dist_grid
    amplitude_weight = amplitude_weight * (array_antenna_theta.size / amplitude_weight.sum(axis=0)).reshape((1, amplitude_weight.shape[1], amplitude_weight.shape[2]))


    beamforming_phases = np.exp(-1j * 2 * np.pi * dist_grid * f / 3e8)
    
    if antispiral:
        
        grid_angles = np.arctan2(ydist, xdist)
        
        antispiral_phase = np.exp(-1j * grid_angles)
        return np.matmul((amplitude_weight * beamforming_phases * antispiral_phase).T, array_data[:, nch, :])
    else:
        return np.matmul((amplitude_weight * beamforming_phases).T, array_data[:, nch, :])



In [ ]:
os.listdir(DIGDATAPATH)

In [ ]:
os.listdir(VNACORRECTION)

In [ ]:
dig_data_file = h5py.File(os.path.join(DIGDATAPATH, '211213_cres2_meas_3ch.h5'), 'r')

In [ ]:
vna_correction_file = h5py.File(os.path.join(VNACORRECTION, '220114_cres2_vna_correction.h5'), 'r')

In [ ]:
print(vna_correction_file['range3cm_step1mm_z0mm']['amplitude'].shape)

In [ ]:
sns.set_theme(context='paper')
acq = 'range3cm_step1mm_z-5mm'
radial_index = 0
frequency_index = 800
roll_size=0
clist = sns.color_palette('deep')

channels = np.arange(0, 120, 2)

#print(channels.size)

for channel_index in range(dig_data_file[acq][f'{radial_index}'].shape[1]):
    
    if channel_index == 0:
        continue
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    
    power = (abs(dig_data_file[acq][f'{radial_index}'][:, channel_index, :]) ** 2).mean(axis=-1)
    vna_correction = np.roll(vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], roll_size)
    
    #ax.plot(power / power.max(), color=clist[0])
    
    #fig = plt.figure(figsize=(13, 8))
    #ax2 = ax.twinx()
    #ax2.plot(vna_correction, color=clist[1])
    #plt.show()

    fig = plt.figure(figsize=(13, 8))
    ax = fig.add_subplot(1,1,1)
    #ax.plot(power * vna_correction)
    ax.plot(10 * np.log10(power / 50e-6))
    #ax2 = ax.twinx()
    
    #ax2.plot(1 / vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], color=clist[2])
    #print(abs(np.fft.fft(dig_data_file[acq][f'{radial_index}'][0, channel_index, :])).argmax())
    
    
    
    




In [ ]:
sns.set_theme(context='paper')
acq = 'range3cm_step1mm_z-10mm'
radial_index = 0
frequency_index = 800
roll_size=46
clist = sns.color_palette('deep')

channel_index = 3

#print(channels.size)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for radial_index in range(31):
    
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    
    power = (10 * np.log10((abs(dig_data_file[acq][f'{radial_index}'][:, channel_index, :]) ** 2).mean(axis=-1) / 50e-6))
    #vna_correction = np.roll(vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], roll_size)
    
    #ax.plot(power / power.max(), color=clist[0])
    
    #fig = plt.figure(figsize=(13, 8))
    #ax2 = ax.twinx()
    #ax2.plot(vna_correction, color=clist[1])
    #plt.show()

    
    #ax.plot(power * vna_correction)
    ax.plot(power)
    #ax2 = ax.twinx()
    
    #ax2.plot(1 / vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], color=clist[2])
    #print(abs(np.fft.fft(dig_data_file[acq][f'{radial_index}'][0, channel_index, :])).argmax())
    
    
    
    


In [ ]:
sns.set_theme(context='paper')
acq = 'range3cm_step1mm_z-5mm'
radial_index = 0
frequency_index = 800
roll_list = [0, 29, 59]
vna_roll = []
clist = sns.color_palette('deep')

channels = np.arange(0, 120, 2)

#print(channels.size)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for channel_index in range(dig_data_file[acq][f'{radial_index}'].shape[1]):
    
    if channel_index == 0:
        continue
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    
    power = (abs(dig_data_file[acq][f'{radial_index}'][:, channel_index, :]) ** 2).mean(axis=-1)
    vna_correction = np.roll(vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], roll_size)
    
    #ax.plot(power / power.max(), color=clist[0])
    
    #fig = plt.figure(figsize=(13, 8))
    #ax2 = ax.twinx()
    #ax2.plot(vna_correction, color=clist[1])
    #plt.show()

    
    #ax.plot(power * vna_correction)
    ax.plot(np.roll(power, roll_list[channel_index - 1]))
    #ax2 = ax.twinx()
    
    #ax2.plot(1 / vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], color=clist[2])
    #print(abs(np.fft.fft(dig_data_file[acq][f'{radial_index}'][0, channel_index, :])).argmax())
    
    
#print(channels.size)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for channel_index in range(dig_data_file[acq][f'{radial_index}'].shape[1]):
    
    if channel_index == 0:
        continue
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    
    power = (abs(dig_data_file[acq][f'{radial_index}'][:, channel_index, :]) ** 2).mean(axis=-1)
    vna_correction = vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index]
    
    #ax.plot(power / power.max(), color=clist[0])
    
    #fig = plt.figure(figsize=(13, 8))
    #ax2 = ax.twinx()
    #ax2.plot(vna_correction, color=clist[1])
    #plt.show()

    
    ax.plot(np.roll(np.flip(1 / vna_correction), 35))
    #ax.plot(np.roll(np.flip(vna_correction), 29))
    #ax.plot(10 * np.log10(np.roll(power, roll_list[channel_index - 1]) / 50e-6))
    #ax2 = ax.twinx()
    
    #ax2.plot(1 / vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], color=clist[2])
    #print(abs(np.fft.fft(dig_data_file[acq][f'{radial_index}'][0, channel_index, :])).argmax())
    
    
    
#print(channels.size)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for channel_index in range(dig_data_file[acq][f'{radial_index}'].shape[1]):
    
    if channel_index == 0:
        continue
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    
    power = (abs(dig_data_file[acq][f'{radial_index}'][:, channel_index, :]) ** 2).mean(axis=-1)
    vna_correction = vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index]
    
    #ax.plot(power / power.max(), color=clist[0])
    
    #fig = plt.figure(figsize=(13, 8))
    #ax2 = ax.twinx()
    #ax2.plot(vna_correction, color=clist[1])
    #plt.show()

    
    ax.plot(np.roll(np.flip(vna_correction), 35) * np.roll(power, roll_list[channel_index - 1]))
    #ax.plot(np.roll(np.flip(vna_correction), 29))
    #ax.plot(10 * np.log10(np.roll(power, roll_list[channel_index - 1]) / 50e-6))
    #ax2 = ax.twinx()
    
    #ax2.plot(1 / vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], color=clist[2])
    #print(abs(np.fft.fft(dig_data_file[acq][f'{radial_index}'][0, channel_index, :])).argmax())
    
    
    
    

# try a soft correction

In [ ]:
## sns.set_theme(context='paper')
acq = 'range3cm_step1mm_z-5mm'
radial_index = 0
frequency_index = 800
roll_list = [0, 29, 59]
roll_size=46
vna_roll = []
clist = sns.color_palette('deep')

channels = np.arange(0, 120, 2)

#print(channels.size)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for channel_index in range(dig_data_file[acq][f'{radial_index}'].shape[1]):
    
    if channel_index == 0:
        continue
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    
    power = (abs(dig_data_file[acq][f'{radial_index}'][:, channel_index, :]) ** 2).mean(axis=-1)
    vna_correction = np.roll(vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], roll_size)
    
    #ax.plot(power / power.max(), color=clist[0])
    
    #fig = plt.figure(figsize=(13, 8))
    #ax2 = ax.twinx()
    #ax2.plot(vna_correction, color=clist[1])
    #plt.show()

    
    #ax.plot(power * vna_correction)
    ax.plot(np.roll(power, roll_list[channel_index - 1]))
    #ax2 = ax.twinx()
    
    #ax2.plot(1 / vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], color=clist[2])
    #print(abs(np.fft.fft(dig_data_file[acq][f'{radial_index}'][0, channel_index, :])).argmax())
    


kernel_size = 3
std = 3
smooth_kernel = scipy.signal.windows.boxcar(kernel_size) / kernel_size


for channel_index in range(dig_data_file[acq][f'{radial_index}'].shape[1]):
    
    if channel_index == 0:
        continue
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    
    power = (abs(dig_data_file[acq][f'{radial_index}'][:, channel_index, :]) ** 2).mean(axis=-1)
    vna_correction = vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index]
    
    #vna_correction = scipy.signal.convolve(vna_correction, smooth_kernel, mode='same')
    #vna_correction = scipy.signal.savgol_filter(vna_correction, 13, 1, deriv=0)
    fig = plt.figure(figsize=(13, 8))
    ax = fig.add_subplot(1,1,1)
    
    ax.plot(np.roll(np.flip(1/ vna_correction), 30), '.')
    
    #vna_correction = scipy.signal.cspline1d_eval(scipy.signal.cspline1d(vna_correction, lamb=30), np.linspace(0, 120, 120))
    #ax.plot(power / power.max(), color=clist[0])
    
    #fig = plt.figure(figsize=(13, 8))
    #ax2 = ax.twinx()
    #ax2.plot(vna_correction, color=clist[1])
    #plt.show()
    
    #ax.plot(np.roll(np.flip(vna_correction), 20))
    
    #fig = plt.figure(figsize=(13, 8))
    #ax = fig.add_subplot(1,1,1)
    #ax.plot(np.roll(np.flip(vna_correction), 29))
    #ax.plot(10 * np.log10(np.roll(power, roll_list[channel_index - 1]) / 50e-6))
    #ax2 = ax.twinx()
    
    #ax2.plot(1 / vna_correction_file[acq]['amplitude'][radial_index, :, frequency_index], color=clist[2])
    #print(abs(np.fft.fft(dig_data_file[acq][f'{radial_index}'][0, channel_index, :])).argmax())
    
    
    fig = plt.figure(figsize=(13, 8))
    ax = fig.add_subplot(1,1,1)
    ax.plot(power * np.roll(np.flip(vna_correction), 30))
    ax.plot(scipy.signal.convolve(power * np.roll(np.flip(vna_correction), 30), smooth_kernel))
